In [86]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [112]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [113]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [151]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
cpi = percentchange(data_source[:CPI], :log) 

real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi

excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)

dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])

term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq

#data = merge(excess_returns, real_t_bill, dy, term, log1p.(data_source[:T_Bill]))
data = merge(excess_returns, real_t_bill, dy, term, cpi)

start = Date(1975, 01,01)
end_d = Date(2020, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq)) 

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.0888,0.4823,-0.0054,0.1074,-1.0278,-0.7566,-0.1025,0.0667,2.5531,-0.0611,0.0108
T_Bill_CPI,0.015,0.1307,0.0309,0.0486,-0.0375,0.3422,0.0077,0.9012,-0.0493,2.0596,0.0331
Dividend_Stock_Price,0.8622,-11.0055,-14.6673,-13.2532,-18.1228,0.2412,-15.9707,0.9917,-1.0111,-17.0108,-15.247
Yeld_10Y_T_Bill,0.0061,0.0413,0.0173,0.027,-0.0306,-0.5952,0.0086,0.9299,0.3178,2.8179,0.0187
CPI,0.0073,0.0604,0.0117,0.0197,-0.0774,-0.2371,0.0036,0.6145,3.1846,1.611,0.0111


nothing

In [153]:
assets_names = colnames(s_data)
T, n_assets = size(s_data)
model = VARModel(s_data)
model_summary(model)

,const,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI,std
Total_Stock_T_Bill,0.1031,0.0469,-1.6374,0.0214,-1.3114,-2.6652,0.0431
T_Bill_CPI,-0.0083,-0.0006,0.9817,-0.0017,0.1962,0.5238,0.0028
Dividend_Stock_Price,-0.0786,-0.0455,0.5203,0.9826,0.8716,1.6574,0.0438
Yeld_10Y_T_Bill,0.0015,0.0012,-0.0131,0.0003,0.9144,-0.0106,0.0011
CPI,0.0073,0.002,0.018,0.0015,-0.1419,0.5201,0.0028


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
Total_Stock_T_Bill,1.0,-0.17,-0.99,0.03,0.14
T_Bill_CPI,-0.17,1.0,0.17,-0.2,-0.94
Dividend_Stock_Price,-0.99,0.17,1.0,-0.02,-0.14
Yeld_10Y_T_Bill,0.03,-0.2,-0.02,1.0,-0.05
CPI,0.14,-0.94,-0.14,-0.05,1.0


In [155]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,0.0613,-0.0107,-16.6829,0.0096,0.0116
5,0.0387,-0.0069,-16.7366,0.0132,0.0064
15,0.0337,-0.0067,-16.8292,0.0159,0.0045
25,0.0324,-0.0069,-16.902,0.0165,0.004


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,0.0742,0.007,0.1635,0.0039,0.007
5,0.0345,0.0073,0.399,0.007,0.0052
15,0.0226,0.0105,0.6095,0.0065,0.0041
25,0.0218,0.0124,0.7148,0.0058,0.0039


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,-0.0466,-0.0,0.0241,-0.0105,0.007
5,0.0072,-0.0065,0.0019,0.0154,-0.0193
15,-0.0319,0.0389,-0.0557,0.0162,0.0092
25,-0.047,0.0598,-0.0374,-0.0263,0.0193


,Total_Stock_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,CPI
1,-0.0449,-0.018,-0.0491,-0.0205,0.0292
5,-0.0065,-0.0026,-0.108,0.0654,-0.0429
15,-0.045,-0.0849,-0.0031,0.0004,0.1475
25,-0.0001,0.0034,-0.028,-0.0566,0.0794


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1127,-0.0841,-0.0628,0.0109,0.0624,0.1116,0.1807,0.2044,0.2329
5,-0.0429,-0.0276,-0.0174,0.0151,0.0388,0.0617,0.0961,0.1054,0.1185
15,-0.0197,-0.0108,-0.0032,0.0184,0.0338,0.0493,0.0708,0.0778,0.0855
25,-0.0191,-0.0103,-0.0033,0.0178,0.0325,0.0473,0.0684,0.0749,0.0826


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0268,-0.0242,-0.0221,-0.0154,-0.0106,-0.0059,0.0008,0.0031,0.0057
5,-0.0239,-0.0212,-0.0189,-0.0117,-0.0069,-0.0021,0.0051,0.0073,0.01
15,-0.0307,-0.0271,-0.0241,-0.0138,-0.0067,0.0004,0.0107,0.0139,0.0177
25,-0.0351,-0.0313,-0.0272,-0.0153,-0.007,0.0014,0.0135,0.0176,0.0229


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-17.0587,-16.9999,-16.948,-16.7954,-16.6835,-16.5723,-16.4133,-16.3638,-16.3017
5,-17.6578,-17.5071,-17.3787,-17.0143,-16.7347,-16.4644,-16.0766,-15.9603,-15.821
15,-18.269,-18.0457,-17.8532,-17.243,-16.8221,-16.4178,-15.8432,-15.6577,-15.4386
25,-18.5775,-18.3289,-18.1074,-17.3772,-16.8939,-16.4182,-15.7344,-15.5281,-15.2794


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,0.0003,0.0018,0.0031,0.0069,0.0095,0.0123,0.0161,0.0173,0.0187
5,-0.0029,-0.0004,0.0018,0.0086,0.0132,0.0179,0.0247,0.0269,0.0294
15,0.0008,0.0032,0.0051,0.0114,0.0159,0.0202,0.0265,0.0287,0.0312
25,0.0031,0.005,0.0068,0.0126,0.0165,0.0204,0.0261,0.0278,0.0297


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0048,-0.0022,0.0001,0.007,0.0117,0.0163,0.0232,0.0253,0.0278
5,-0.0057,-0.0038,-0.0022,0.0029,0.0065,0.01,0.015,0.0165,0.0185
15,-0.0049,-0.0033,-0.0021,0.0018,0.0045,0.0073,0.0112,0.0124,0.0141
25,-0.0051,-0.0037,-0.0024,0.0014,0.0039,0.0066,0.0103,0.0115,0.0131


In [142]:
real_equity_scenarios = reshape(scenarios[1, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)
real_bond_scenarios = reshape(scenarios[2, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)

ret_in_years = cum_returns_in_periods(vcat(real_equity_scenarios, real_bond_scenarios), periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names[1:2], string.(periods))

for a in 1:2
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  



,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0339,0.0304
5,0.0222,0.0034
15,0.0173,0.0015
25,0.0155,0.0019


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0717,0.0391
5,0.0311,0.0171
15,0.0158,0.0072
25,0.0111,0.0049


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0117,-0.0548
5,0.0316,0.0316
15,0.0218,0.009
25,-0.0152,0.0211


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.0316,0.0417
5,0.0896,-0.0271
15,0.0629,0.0114
25,0.0045,-0.0133


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.134,-0.1058,-0.0843,-0.0139,0.0342,0.0819,0.1522,0.176,0.2042
5,-0.0483,-0.0377,-0.0287,0.0013,0.0225,0.043,0.073,0.0835,0.095
15,-0.0193,-0.0135,-0.0085,0.0069,0.0172,0.0278,0.0432,0.0481,0.0533
25,-0.0109,-0.0065,-0.0028,0.0081,0.0155,0.0229,0.0338,0.0374,0.0414


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0619,-0.0468,-0.0347,0.0044,0.0313,0.0567,0.0939,0.106,0.1208
5,-0.0356,-0.0297,-0.0244,-0.0082,0.0032,0.015,0.0316,0.0373,0.0436
15,-0.015,-0.0128,-0.0106,-0.0034,0.0015,0.0063,0.0133,0.0155,0.0182
25,-0.0094,-0.0076,-0.006,-0.0014,0.0019,0.0052,0.01,0.0115,0.0132


In [141]:
mdd, mddl = max_drawdown_and_length(real_equity_scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(real_bond_scenarios[1,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.15603  0.219442  0.268403  0.329568  0.451828

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 17.0  29.0  41.0  57.0  88.0

:Price_10Y_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.124018  0.174967  0.212861  0.25475  0.340629

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 28.0  46.0  61.0  79.0  98.0